In [64]:
import shutil
from PIL import Image
import os
import json

from pycocotools import mask as maskUtils
import cv2

def rle2polygon(segmentation, h, w):
    """
    将游程编码(RLE)或多边形分割数据转换为多边形坐标列表。
    此功能对于处理图像数据集中的不同分割格式至关重要。

    参数:
        segmentation (dict or list): RLE或多边形分割数据。
        h (int): 图像的高度。
        w (int): 图像的宽度。

    返回:
        list: 表示为 [x1, y1, x2, y2, ..., xn, yn] 的多边形列表。

    详细信息:
    该函数首先检查分割是否为RLE格式。如果, 则将RLE解码为二进制掩码。
    如果是多边形, 则先将其转换为RLE, 然后解码。
    然后使用OpenCV查找轮廓(代表对象的边缘)并将这些轮廓近似为多边形。
    """
    # Check if segmentation is in RLE format (dict)
    if isinstance(segmentation, dict):
        m = maskUtils.decode(segmentation)
    else:
        # Convert polygon segmentation to RLE first
        rle = maskUtils.frPyObjects(segmentation, h, w)
        compressed_rle = maskUtils.merge(rle)
        m = maskUtils.decode(compressed_rle)

    m[m > 0] = 255
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    
    for contour in contours:
        epsilon = 0.001 * cv2.arcLength(contour, True)
        contour_approx = cv2.approxPolyDP(contour, epsilon, True)
        polygon = contour_approx.flatten().tolist()
        polygons.append(polygon)
    return polygons

def crop_img(image_path, output_folder, dataset_name, annotation, adjusted_segmentation):
    """
    根据预定义坐标裁剪图像, 并相应调整其分割标注。
    此功能用于专注于图像及其相应注释的特定部分。

    参数:
        image_path (str): 源图像的路径。
        output_folder (str): 裁剪图像将被保存的目录。
        dataset_name (str): 用于命名输出文件的数据集名称。
        annotation (dict): 图像的原始注释数据。
        adjusted_segmentation (list): 转换为多边形坐标的分割数据。

    返回:
        tuple: 更新的注释，新图像高度和宽度。

    详细信息:
    此函数打开图像，根据预定义坐标进行裁剪，然后保存裁剪后的图像。
    它还调整注释中的分割坐标，以匹配裁剪后图像的新尺寸。
    """
    # Load the image
    image = Image.open(image_path)

    # Get the dimensions of the image
    width, height = image.size

    # Define the coordinates for cropping
    left = 390
    right = width - 90
    top = 55
    bottom = height - 55

    # Ensure coordinates are within the image dimensions
    left = max(0, left)
    right = min(width, right)
    top = max(0, top)
    bottom = min(height, bottom)

    # Crop the image
    cropped_image = image.crop((left, top, right, bottom))

    # Save the cropped image
    image_filename = os.path.basename(image_path)
    output_folder = output_folder + '\images'

    new_image_filename = dataset_name+'_'+image_filename
    output_image_path = os.path.join(output_folder, new_image_filename)
    cropped_image.save(output_image_path)

    # Adjust the segmentation coordinates
    new_segmentation = []
    for segment in adjusted_segmentation:
        adjusted_segment = []
        for i in range(0, len(segment), 2):
            x, y = segment[i], segment[i + 1]
            # Adjusting the coordinates
            x -= left
            y -= top
            # Ensure coordinates are within the new image dimensions
            x = max(0, min(x, right - left))
            y = max(0, min(y, bottom - top))
            adjusted_segment.extend([x, y])
        new_segmentation.append(adjusted_segment)
    annotation['segmentation'] = new_segmentation

    new_width =  right - left
    new_height = bottom - top
    return annotation,new_height,new_width


def process_dataset(annotations, images, output_folder, dataset_name,h,w,js_path):
    """
    处理每个数据集，通过裁剪图像和更新标注。
    此函数是此项目特定需求的数据集的核心。

    参数:
        annotations (list): 图像的注释列表。
        images (list): 图像数据列表。
        output_folder (str): 已处理图像和注释的目录。
        dataset_name (str): 数据集名称。
        h (int): 图像高度。
        w (int): 图像宽度。
        js_path (str): 对应JSON文件的路径。

    返回:
        tuple: 已处理的图像和更新的注释。

    详细信息:
    对于数据集中的每个注释，此功能查找相应的图像，应用 `crop_img` 函数，
    并更新注释。它确保每个图像只处理一次，并编译已处理图像和其新注释的列表。
    """
    # Dictionary to hold updated annotations
    updated_annotations = []
    # Set to track processed image ids
    processed_image_ids = set()

    # Process each annotation
    for annotation in annotations:
        if 'segmentation' in annotation and annotation['segmentation']:
            image_id = annotation['image_id']
            s1 = rle2polygon(annotation['segmentation'],h,w)
            # Find the corresponding image
            image_data = next((img for img in images if img['id'] == image_id), None)
            if image_data:
                image_path = os.path.join(dataset_name, image_data['file_name'])
                # !!!NOTE:如果有相同图片名，需要自定义数据集名字!!!
                new_dataset_name = js_path.split('\\')[5]
                new_image_filename = new_dataset_name+'_'+image_data['file_name']
                image_data['file_name'] =new_image_filename
                updated_annotation,new_height,new_width = crop_img(
                    image_path, output_folder, new_dataset_name, annotation,s1)
                updated_annotations.append(updated_annotation)
                image_data['height'] = new_height
                image_data['width'] = new_width
                processed_image_ids.add(image_id)

    # Filter images to include only those that were processed
    filtered_images = [img for img in images if img['id'] in processed_image_ids]

    return filtered_images, updated_annotations


def coco_to_yolo(coco_file_path, output_dir):
    """
    将COCO格式的注释转换为YOLO格式, 适用于不同类型的对象检测模型。

    参数:
        coco_file_path (str): COCO格式文件的路径。
        output_dir (str): 保存YOLO格式注释的目录。

    详细信息:
    此功能读取COCO数据集, 迭代其注释, 并将每个注释转换为YOLO格式。
    它为每个图像创建一个文本文件, 其中每行对应一个以YOLO格式表示的对象。
    该函数考虑图像尺寸以标准化边界框的坐标。
    """
    # Load the COCO data
    with open(coco_file_path, "r", encoding='utf-8') as file:
        coco_data = json.load(file)

    # Create a directory to store the YOLO formatted annotations
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a mapping from image id to file name and its dimensions for easy access
    image_id_to_info = {img["id"]: (img["file_name"], img["width"], img["height"]) for img in coco_data["images"]}

    # Create a mapping from category id to category index
    category_id_to_index = {cat["id"]: idx for idx, cat in enumerate(coco_data["categories"])}

    # Iterate over each annotation and save in YOLO format
    for annotation in tqdm(coco_data["annotations"], desc="Converting annotations"):
        # Get file name, width, and height for the annotation's image
        image_filename, img_width, img_height = image_id_to_info[annotation["image_id"]]
        txt_filename = os.path.splitext(image_filename)[0] + ".txt"
        txt_filepath = os.path.join(output_dir, txt_filename)
        
        # Check if the file already exists
        if os.path.exists(txt_filepath):
            print(f"File already exists: {txt_filepath}")

        # Get the class index for the annotation
        class_index = category_id_to_index[annotation["category_id"]]

        # Normalize segmentation points
        normalized_points = []
        segmentation = annotation["segmentation"][0]  # Assuming each annotation has one segmentation
        for i in range(0, len(segmentation), 2):
            normalized_x = segmentation[i] / img_width
            normalized_y = segmentation[i + 1] / img_height
            normalized_points.extend([normalized_x, normalized_y])

        # Write to the txt file
        with open(txt_filepath, "a") as file:
            file.write(f"{class_index} {' '.join(map(str, normalized_points))}\n")

# Function to create a new dataset
def create_new_dataset(original_dataset, processed_images, processed_annotations):
    return {
        "info": original_dataset.get("info", {}),
        "licenses": original_dataset.get("licenses", []),
        "categories": original_dataset.get("categories", []),
        "images": processed_images,
        "annotations": processed_annotations
    }

def setup_directories(output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    else:
        shutil.rmtree(output_folder)
        os.makedirs(output_folder)
    if not os.path.exists(os.path.join(output_folder, 'images')):
        os.makedirs(os.path.join(output_folder, 'images'))
    if not os.path.exists(os.path.join(output_folder, 'labels')):
        os.makedirs(os.path.join(output_folder, 'labels'))

def get_dataset_name(js_path):
    return js_path.split('\\')[5]

def load_dataset(js_path):
    with open(js_path, encoding='utf-8') as f:
        dataset = json.load(f)
    return dataset, dataset['images'], dataset['annotations']

def get_image_dimensions(images):
    if images:
        return images[0]['height'], images[0]['width']
    return 0, 0

def save_new_dataset(dataset, output_folder, dataset_name):
    tmp_path = os.path.join(output_folder, 'tmp')
    if not os.path.exists(tmp_path):
        os.makedirs(tmp_path)
    else:
        shutil.rmtree(tmp_path)
        os.makedirs(tmp_path)
    output_file = os.path.join(tmp_path, f'{dataset_name}_processed.json')
    with open(output_file, 'w') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)
    return output_file


# 主函数
def main():
    # Step1: 设置image和对应json的路径名
    json_files = [r'C:\Users\z8443\Downloads\手术器械标注\20230811_110050\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230810_092218\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230802_103503\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230728_092252\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230728_083801\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230714_123805\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230713_092647\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230714_113213\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230714_104658\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230714_093927\annotations\instances_default.json',
                  r'C:\Users\z8443\Downloads\手术器械标注\20230713_103330\annotations\instances_default.json',
                  ]
    image_files = [r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230811_110050',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230810_092218',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230802_103503',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230728_092252',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230728_083801',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230714_123805',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230713_092647',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230714_113213',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230714_104658',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230714_093927',
                    r'C:\Users\z8443\Downloads\pic\TV_CAM_device_20230713_103330',
                    ]
    
    # 自定义新的数据集名
    output_folder = 'new_dataset'
    setup_directories(output_folder)
    
    # Step2: 针对每一个json和对应的image：1.进行图片裁剪 2.rle_mask坐标转换为coco
    for js_path, dataset_name in zip(json_files, image_files):
        new_dataset_name = get_dataset_name(js_path)
        original_dataset, images, annotations = load_dataset(js_path)
        h, w = get_image_dimensions(images)

        processed_images, processed_annotations = process_dataset(
            annotations, images, output_folder, dataset_name, h, w,js_path)

        new_dataset = create_new_dataset(
            original_dataset, processed_images, processed_annotations)

        coco_json_path = save_new_dataset(new_dataset, output_folder, new_dataset_name)
        
        # Step3: 将coco格式的标注转换为yolo格式
        coco_to_yolo(coco_json_path, os.path.join(output_folder, 'labels'))
        shutil.rmtree(os.path.join(output_folder, 'tmp'))
        
if __name__ == "__main__":
    main()

Converting annotations: 100%|██████████| 67/67 [00:00<00:00, 656.90it/s]


# CVAT的COCO数据到YOLO格式转换

## 描述
此脚本用于处理图像数据集，以用于对象检测任务。它涉及裁剪图像、调整尺寸、调整注释，并将数据集从COCO格式转换为YOLO格式以进行训练。

## 工作流程
1. 加载所需的JSON和图像文件。
2. 创建处理数据的必要目录。
3. 处理每个JSON文件：裁剪图像，调整图像尺寸和注释。
4. 以COCO格式保存处理后的数据。
5. 将COCO格式数据转换为YOLO格式，以进行进一步的训练。

## 功能
- `rle2polygon`: 将RLE或多边形分割数据转换为多边形坐标。
- `crop_img`: 裁剪图像并调整其注释。
- `process_dataset`: 处理每个数据集，更新注释和图像。
- `create_new_dataset`: 使用处理过的数据创建一个新数据集。
- `coco_to_yolo`: 将COCO格式注释转换为YOLO格式。

## 使用方法
在设置好 `json_files` 和 `image_files` 列表与数据集路径后，运行脚本。确保所有依赖都已安装并正确设置路径。

## 依赖
- Python 3.x
- 库：shutil, os, json, cv2, PIL, pycocotools

## 注意事项
- 根据需要自定义裁剪坐标和其他参数。
- 确保已安装正确的Python版本和依赖。
- 脚本假设JSON文件路径具有特定结构，以提取数据集名称。
